# **Importing Libraries**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# pip install tensorflow==2.18.0 keras==3.6.0

In [3]:
import numpy as np
import pandas as pd
# import keras_tuner as kt
from matplotlib import pyplot as plt
import tensorflow as tf
from prophet import Prophet
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError, Huber
from tensorflow.keras.models import load_model
from keras.models import Sequential
from keras.layers import InputLayer, Dense, LSTM, Dropout,GRU
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, root_mean_squared_error
import seaborn as sns
import os
import datetime
import cv2

In [ ]:
print(tf.__version__)
print(keras.__version__)

2.18.0
3.6.0


# **Reading CSV, Creat DataFrame**

In [7]:
# base_dir = '/content/drive/MyDrive/MTech Internship/interpolated_file.csv'
folder_path = (r"/content/drive/MyDrive/MTech Internship/bopal_solar_insolation_2016_2022_final.csv")
df = pd.read_csv(folder_path)
# print(df.head(20))

df['date_time'] = pd.to_datetime(df['date_time'], format = 'mixed')
df = df[(df['date_time'].dt.time >= pd.to_datetime('7:00').time()) &
                  (df['date_time'].dt.time <= pd.to_datetime('19:30').time())]
print(df.head(100))
print(df.shape)
# print(df.iloc[50:100])
# TESTING

folder_path_t = (r"/content/drive/MyDrive/MTech Internship/April_2024.csv")
df_t = pd.read_csv(folder_path_t)

df_t['date_time'] = pd.to_datetime(df_t['date_time'], format = 'mixed')
df_t = df_t[(df_t['date_time'].dt.time >= pd.to_datetime('7:00').time()) &
                  (df_t['date_time'].dt.time <= pd.to_datetime('19:30').time())]
# df_t = df_t.head(260)
print(df_t.head(20))
print(df_t.shape)

             date_time      ghi
0  2016-01-01 07:00:00    0.000
1  2016-01-01 07:30:00    0.000
2  2016-01-01 08:00:00    0.000
3  2016-01-01 08:30:00    0.000
4  2016-01-01 09:00:00   63.336
..                 ...      ...
95 2016-01-04 15:30:00  376.831
96 2016-01-04 16:00:00  322.086
97 2016-01-04 16:30:00  193.469
98 2016-01-04 17:00:00  127.113
99 2016-01-04 17:30:00   25.190

[100 rows x 2 columns]
(66482, 2)
             date_time        ghi
14 2024-01-04 07:00:00   89.90500
15 2024-01-04 07:30:00  157.20000
16 2024-01-04 08:00:00  177.67000
17 2024-01-04 08:30:00  307.31833
18 2024-01-04 09:00:00  352.46167
19 2024-01-04 09:30:00  597.24164
20 2024-01-04 10:00:00  725.10500
21 2024-01-04 10:30:00  814.79170
22 2024-01-04 11:00:00  739.87164
23 2024-01-04 11:30:00  864.28330
24 2024-01-04 12:00:00  963.52167
25 2024-01-04 12:30:00  976.79000
26 2024-01-04 13:00:00  974.26330
27 2024-01-04 13:30:00  930.86334
28 2024-01-04 14:00:00  913.39830
29 2024-01-04 14:30:00  854.14500
30 

In [ ]:
# df_t = pd.DataFrame(df_t)


# # Filter the data to include only entries from Jan 1st to Jan 10th
# start_date = '01-01-2024'
# end_date = '01-11-2024'
# mask = (df_t['date_time'] >= start_date) & (df_t['date_time'] < end_date)
# filtered_data = df.loc[mask]

# # Display the filtered data
# print(filtered_data)

# **Preprocessing**

In [8]:
starting_timestamp = "2016/01/01 0:00"
ending_timestamp = "2022/12/31 23:30"
array_range = pd.date_range(start=starting_timestamp, end=ending_timestamp, freq='30T')

array_timestamps = []

start_time = pd.to_datetime("7:00 AM").time()
end_time = pd.to_datetime("7:00 PM").time()
max_end_time = pd.to_datetime("7:30 PM").time()

for timestamp in array_range:
    if timestamp.time() >= start_time and timestamp.time() <= end_time:
        array_timestamps.append(timestamp)
    elif timestamp.time() > end_time and timestamp.time() <= max_end_time:
        array_timestamps.append(timestamp)


df_date = pd.DataFrame({'date_time':array_timestamps})
df_date.set_index('date_time')

p_df = pd.merge(df_date, df, on ='date_time', how='left')
# print(p_df.head(40))

cols = list(p_df)[1]
print(cols)

df_for_training = p_df[cols].astype(float)
        # print(df_for_training.head(10))
df_for_training = df_for_training.dropna()
# df_for_training.to_csv(os.path.join(base_dir,"interpolated_file.csv"),header=True)
print(df_for_training.head(40))
print(df_for_training.shape)
df_for_training = np.array(df_for_training).reshape(-1,1)




# TESTING

starting_timestamp_t = "04/01/2024 0:00"
ending_timestamp_t = "05/15/2024 23:30"
array_range_t = pd.date_range(start=starting_timestamp_t, end=ending_timestamp_t, freq='30T')

array_timestamps_t = []
start_time_t = pd.to_datetime("7:00 AM").time()
end_time_t = pd.to_datetime("7:00 PM").time()
max_end_time_t = pd.to_datetime("7:30 PM").time()

for timestamp_t in array_range_t:
    if timestamp_t.time() >= start_time_t and timestamp_t.time() <= end_time_t:
        array_timestamps_t.append(timestamp_t)
    elif timestamp_t.time() > end_time_t and timestamp_t.time() <= max_end_time_t:
        array_timestamps_t.append(timestamp_t)

df_date_t = pd.DataFrame({'date_time':array_timestamps_t})
df_date_t.set_index('date_time')
p_df_t = pd.merge(df_date_t, df_t, on ='date_time', how='left')

cols_t = list(df_t)[1]
print(cols)
df_for_testing = df_t[cols].astype(float)
        # print(df_for_testing.head(10))
df_for_testing = df_for_testing.dropna()
# df_for_testing.to_csv(os.path.join(base_dir,"interpolated_file.csv"),header=True)
print(df_for_testing.head(60))
# print(p_df_t.head(150))
print(df_for_testing.shape)
df_for_testing = np.array(df_for_testing).reshape(-1,1)


<ipython-input-8-555347fee03b>:3: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  array_range = pd.date_range(start=starting_timestamp, end=ending_timestamp, freq='30T')


ghi
0       0.000
1       0.000
2       0.000
3       0.000
4      63.336
5      85.460
6     232.009
7     261.788
8     344.058
9     426.328
10    533.932
11    596.620
12    593.633
13    596.862
14    574.698
15    516.912
16    476.647
17    371.110
18    315.488
19    187.728
20    120.578
21     21.805
22      0.000
23      0.000
24      0.000
25      0.000
26      0.000
27      0.000
28      0.000
29     42.220
30     63.555
31    197.796
32    231.993
33    373.987
34    445.619
35    517.250
36    531.404
37    596.288
38    594.129
39    596.755
Name: ghi, dtype: float64
(66482,)
ghi
14      89.905000
15     157.200000
16     177.670000
17     307.318330
18     352.461670
19     597.241640
20     725.105000
21     814.791700
22     739.871640
23     864.283300
24     963.521670
25     976.790000
26     974.263300
27     930.863340
28     913.398300
29     854.145000
30     802.996640
31     712.960000
32     612.425000
33     497.531680
34     350.325000
35     255.501660
3

<ipython-input-8-555347fee03b>:42: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  array_range_t = pd.date_range(start=starting_timestamp_t, end=ending_timestamp_t, freq='30T')


In [ ]:
max_value = p_df['ghi'].max()
min_value = p_df['ghi'].min()
print(max_value,min_value)

1001.75 0.0


# **Handling Missing Values**


In [ ]:

def missingValUsingMedian(missing_val_date):

    print("Neighbouring values not available")
    print("Find previous 3 days values and perform median operation")

    dt = missing_val_date
    prev_dates_lst = []
    prev_values_lst = []
    for k in range(3):
        dt1 = dt - datetime.timedelta(days=k+1)
        dt_str = dt1.strftime("%Y-%m-%d %H:%M")
        prev_dates_lst.append(dt_str)
        value = df[df['date_time'] == dt_str][col].values[0]
        if value != None:
            print(value)
            prev_values_lst.append(value)
    print(prev_values_lst)
    missing_val = np.nanmedian(prev_values_lst)
    print(missing_val)

    return missing_val

for col in p_df.columns:
    print(col)
    for i, row in p_df.iterrows():
        dt = row[0]
        if i==0:
            pass
        else:
            if pd.isnull(p_df.loc[i][col]):
                print("Value null for dt::",row[0])
                if not pd.isnull(p_df.loc[i-1][col]):
                    if not pd.isnull(p_df.loc[i+1][col]):
                        p_df.loc[i,col] = (p_df.loc[i-1][col] + p_df.loc[i+1][col])/2
                        print(p_df.loc[i, col])
                        # p_df[col].replace(p_df.loc[i,col], new_val, inplace = True)
                        print("Replaced Value:", p_df.loc[i, col])

                    elif not pd.isnull(p_df.loc[i+2][col]):
                        print("here--1")
                        p_df.loc[i,col] = (p_df.loc[i-1][col] + p_df.loc[i+2][col])/2
                        # p_df.replace(p_df.loc[i,col], new_val, inplace = True)
                    else:
                        replaced_val = missingValUsingMedian(dt)
                        print("missing Values::",replaced_val)
                        p_df.loc[i,col]= replaced_val
                        p_df.replace(p_df.loc[i,col], replaced_val, inplace = True)
                        print("Replaced Value:", p_df.loc[i, col])
                else:
                    replaced_val = missingValUsingMedian(dt)
                    print("missing Values::",replaced_val)
                    p_df.loc[i,col]= replaced_val
                    # p_df.replace(p_df.loc[i,col], replaced_val, inplace = True)
                    print("Replaced Value:", p_df.loc[i, col])



date_time


<ipython-input-5-8cfcb58865e1>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dt = row[0]


ghi
Value null for dt:: 2022-01-04 07:00:00
Neighbouring values not available
Find previous 3 days values and perform median operation
0.455
0.5183333
0.535
[0.455, 0.5183333, 0.535]
0.5183333
missing Values:: 0.5183333
Replaced Value: 0.5183333
Value null for dt:: 2022-01-04 07:30:00
Neighbouring values not available
Find previous 3 days values and perform median operation
18.868334
22.358334
11.283334
[18.868334, 22.358334, 11.283334]
18.868334
missing Values:: 18.868334
Replaced Value: 18.868334
Value null for dt:: 2022-01-04 08:00:00
Neighbouring values not available
Find previous 3 days values and perform median operation
59.55
70.66
31.163334
[59.55, 70.66, 31.163334]
59.55
missing Values:: 59.55
Replaced Value: 59.55
Value null for dt:: 2022-01-04 08:30:00
Neighbouring values not available
Find previous 3 days values and perform median operation
141.57
145.96
160.01666
[141.57, 145.96, 160.01666]
145.96
missing Values:: 145.96
Replaced Value: 145.96
Value null for dt:: 2022-01-0

<ipython-input-5-8cfcb58865e1>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dt = row[0]
<ipython-input-5-8cfcb58865e1>:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Value null for dt::",row[0])
<ipython-input-5-8cfcb58865e1>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dt = row[0]
<ipython-input-5-8cfcb58865e1>:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, i

0.0
45.913334
45.461666
[0.0, 45.913334, 45.461666]
45.461666
missing Values:: 45.461666
Replaced Value: 45.461666
Value null for dt:: 2022-01-04 18:00:00
Neighbouring values not available
Find previous 3 days values and perform median operation
0.0
5.6466665
5.3766665
[0.0, 5.6466665, 5.3766665]
5.3766665
missing Values:: 5.3766665
Replaced Value: 5.3766665
Value null for dt:: 2022-01-04 18:30:00
Neighbouring values not available
Find previous 3 days values and perform median operation
0.0
0.0
0.0
[0.0, 0.0, 0.0]
0.0
missing Values:: 0.0
Replaced Value: 0.0
Value null for dt:: 2022-01-04 19:00:00
here--1
Value null for dt:: 2022-01-04 19:30:00
0.24875001
Replaced Value: 0.24875001
Value null for dt:: 2022-01-19 07:00:00
Neighbouring values not available
Find previous 3 days values and perform median operation
0.265
0.27333334
0.3
[0.265, 0.27333334, 0.3]
0.27333334
missing Values:: 0.27333334
Replaced Value: 0.27333334
Value null for dt:: 2022-01-19 07:30:00
Neighbouring values not av

...

Value null for dt:: 2022-01-19 14:00:00
Neighbouring values not available
Find previous 3 days values and perform median operation
710.91
660.505
661.1583
[710.91, 660.505, 661.1583]
661.1583
missing Values:: 661.1583
Replaced Value: 661.1583
Value null for dt:: 2022-01-19 14:30:00
Neighbouring values not available
Find previous 3 days values and perform median operation
656.36
613.98
607.4217
[656.36, 613.98, 607.4217]
613.98
missing Values:: 613.98
Replaced Value: 613.98
Value null for dt:: 2022-01-19 15:00:00
Neighbouring values not available
Find previous 3 days values and perform median operation
583.92334
543.93835
537.995
[583.92334, 543.93835, 537.995]
543.93835
missing Values:: 543.93835
Replaced Value: 543.93835
Value null for dt:: 2022-01-19 15:30:00
Neighbouring values not available
Find previous 3 days values and perform median operation
493.31332
467.30167
450.39667
[493.31332, 467.30167, 450.39667]
467.30167
missing Values:: 467.30167
Replaced Value: 467.30167
Value null

<ipython-input-5-8cfcb58865e1>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dt = row[0]
<ipython-input-5-8cfcb58865e1>:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Value null for dt::",row[0])
<ipython-input-5-8cfcb58865e1>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dt = row[0]
<ipython-input-5-8cfcb58865e1>:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, i

Replaced Value: 755.2833
Value null for dt:: 2022-01-26 12:30:00
Neighbouring values not available
Find previous 3 days values and perform median operation
814.01166
778.43835
728.7833
[814.01166, 778.43835, 728.7833]
778.43835
missing Values:: 778.43835
Replaced Value: 778.43835
Value null for dt:: 2022-01-26 13:00:00
Neighbouring values not available
Find previous 3 days values and perform median operation
815.845
784.9667
726.3883
[815.845, 784.9667, 726.3883]
784.9667
missing Values:: 784.9667
Replaced Value: 784.9667
Value null for dt:: 2022-01-26 13:30:00
Neighbouring values not available
Find previous 3 days values and perform median operation
798.95667
777.0883
727.375
[798.95667, 777.0883, 727.375]
777.0883
missing Values:: 777.0883
Replaced Value: 777.0883
Value null for dt:: 2022-01-26 14:00:00
Neighbouring values not available
Find previous 3 days values and perform median operation
771.0467
734.83
685.3383
[771.0467, 734.83, 685.3383]
734.83
missing Values:: 734.83
Replace

<ipython-input-5-8cfcb58865e1>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dt = row[0]
<ipython-input-5-8cfcb58865e1>:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Value null for dt::",row[0])
<ipython-input-5-8cfcb58865e1>:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  dt = row[0]
<ipython-input-5-8cfcb58865e1>:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, i

IndexError: index 0 is out of bounds for axis 0 with size 0

In [9]:
e = 100   # epochs
n_past=26*7   # Number of past days we want to use to predict the future.
n_future = 26*3   # Number of days we want to look into the future based on the past days.
n_features = 1   # Single variate


# **Making Windows**

In [ ]:

# Empty lists to be populated using formatted training data
dataX, dataY = [], []

for i in range(0,len(df_for_training)-n_past-n_future):
    a = df_for_training[i:(i+n_past), 0]
    dataX.append(a)
    b = df_for_training[i + n_past:i + n_past + n_future, 0]
    dataY.append(b)
dataX, dataY = np.array(dataX), np.array(dataY)

# print(dataX)
print(dataX.shape,dataY.shape)
# print(dataY)

# TESTING

dataX_t, dataY_t = [], []
# print(df_for_testing)
for i in range(0,len(df_for_testing)-n_past-n_future):
    a = df_for_testing[i:(i+n_past), 0]
    dataX_t.append(a)
    b = df_for_testing[i + n_past:i + n_past + n_future, 0]
    dataY_t.append(b)
dataX_t, dataY_t = np.array(dataX_t), np.array(dataY_t)

print(dataX_t.shape,dataY_t.shape)
print(dataX_t)
print(dataY_t)


(66222, 182) (66222, 78)
(520, 182) (520, 78)
[[ 89.905     157.2       177.67      ...  38.806667    1.7716666
    0.       ]
 [157.2       177.67      307.31833   ...   1.7716666   0.
   57.358334 ]
 [177.67      307.31833   352.46167   ...   0.         57.358334
  156.105    ]
 ...
 [ 37.868332    4.6716666   0.        ... 402.12668   255.44333
   47.415    ]
 [  4.6716666   0.         69.335     ... 255.44333    47.415
   59.336666 ]
 [  0.         69.335     105.361664  ...  47.415      59.336666
    4.74     ]]
[[ 57.358334  156.105     268.32834   ...  48.045       3.9533334
    0.       ]
 [156.105     268.32834   390.70834   ...   3.9533334   0.
   62.855    ]
 [268.32834   390.70834   515.705     ...   0.         62.855
  145.37666  ]
 ...
 [ 59.336666    4.74        0.        ... 457.385     326.55334
   62.203335 ]
 [  4.74        0.         93.69666   ... 326.55334    62.203335
   36.993332 ]
 [  0.         93.69666   212.745     ...  62.203335   36.993332
    4.488333 ]]


# **Scaling**

In [ ]:

scalerX = MinMaxScaler(feature_range=(0, 1))
scalerX = scalerX.fit(dataX)
dataX_scaled = scalerX.transform(dataX)
print(dataX_scaled[:10])
counts = np.isnan(dataX_scaled).sum()
print(counts)


scalerY = MinMaxScaler(feature_range=(0, 1))
scalerY = scalerY.fit(dataY)
dataY_scaled = scalerY.transform(dataY)
print(dataY_scaled[:10])
counts = np.isnan(dataY_scaled).sum()
print(counts)

dataX_scaled = dataX_scaled.reshape((dataX_scaled.shape[0], dataX_scaled.shape[1], 1))
dataY_scaled = dataY_scaled.reshape((dataY_scaled.shape[0],dataY_scaled.shape[1], 1))

print(dataX_scaled.shape,dataY_scaled.shape)
# # df_for_training_scaled = (df_for_training - min_value) /(max_value - min_value)
# # print(df_for_training)

# df_for_testing_scaled = scaler.transform(df_for_testing)
# print(df_for_testing_scaled[:10])
# # counts = np.isnan(df_for_training_scaled).sum()


# print(df_for_testing)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.06322536 ... 0.         0.         0.        ]
 ...
 [0.26133067 0.34345695 0.42558323 ... 0.06473771 0.19844273 0.23349538]
 [0.34345695 0.42558323 0.53299925 ... 0.19844273 0.23349538 0.37561867]
 [0.42558323 0.53299925 0.59557774 ... 0.23349538 0.37561867 0.44813177]]
0
[[0.         0.         0.         0.04287896 0.06473771 0.19844273
  0.23349538 0.37561867 0.44813177 0.52064487 0.53762815 0.60121288
  0.59943299 0.60249264 0.58288096 0.55128924 0.48619416 0.43736761
  0.32709858 0.19869229 0.1334874  0.02984377 0.         0.
  0.         0.         0.         0.         0.         0.03673471
  0.05824707 0.18118992 0.21656202 0.35084602 0.42070776 0.4905705
  0.50869978 0.5686309  0.56755278 0.56944946 0.54820065 0.51597005
  0.45256801 0.40195158 0.29644422 0.23134914 0.1138877  0.04091141
  0.         0.    

In [ ]:
scalerX_test = MinMaxScaler(feature_range=(0, 1))
scalerX_test = scalerX_test.fit(dataX_t)
dataX_test_scaled = scalerX_test.transform(dataX_t)
print(dataX_test_scaled[:10])
counts = np.isnan(dataX_test_scaled).sum()
print(counts)


scalerY_test = MinMaxScaler(feature_range=(0, 1))
scalerY_test = scalerY_test.fit(dataY_t)
dataY_test_scaled = scalerY.transform(dataY_t)
print(dataY_test_scaled[:10])
counts = np.isnan(dataY_test_scaled).sum()
print(counts)


dataX_test_scaled = dataX_test_scaled.reshape((dataX_test_scaled.shape[0], dataX_test_scaled.shape[1], 1))
dataY_test_scaled = dataY_test_scaled.reshape((dataY_test_scaled.shape[0], dataY_test_scaled.shape[1], 1))
print(dataX_test_scaled.shape,dataY_test_scaled.shape)

[[0.08708757 0.15227369 0.1721022  ... 0.03690888 0.00168503 0.        ]
 [0.15227369 0.1721022  0.29768763 ... 0.00168503 0.         0.0545533 ]
 [0.1721022  0.29768763 0.34141627 ... 0.         0.0545533  0.14847088]
 ...
 [0.78925787 0.71668564 0.83719851 ... 0.49048509 0.59306238 0.69253598]
 [0.71668564 0.83719851 0.93332696 ... 0.59306238 0.69253598 0.77990212]
 [0.83719851 0.93332696 0.94617949 ... 0.69253598 0.77990212 0.84058681]]
0
[[0.05725813 0.15583229 0.26785959 0.39002579 0.51480409 0.62246732
  0.72687297 0.81857086 0.88226439 0.92800599 0.96626733 1.00021962
  1.00465016 0.9787405  0.94406958 0.90701442 0.8326262  0.74052408
  0.63250146 0.52249563 0.40017302 0.2730588  0.06422593 0.04118459
  0.00266866 0.         0.06481325 0.17024041 0.29260129 0.41612343
  0.53786208 0.65090425 0.75296232 0.84239245 0.90009483 0.95220029
  0.97955744 0.87972378 0.98264038 0.95364113 0.91502035 0.88283169
  0.83348969 0.78038268 0.73239331 0.67515016 0.61176611 0.247427
  0.07696863

In [ ]:
print(df_for_testing.shape)

(780, 1)


# **Train-Test Split**

In [ ]:
# SPLIT = 0.8
# X_train = dataX[:int(SPLIT * len(dataX))]
# y_train = dataY[:int(SPLIT * len(dataY))]
# X_test = dataX[int(SPLIT * len(dataX)):]
# y_test = dataY[int(SPLIT * len(dataY)):]
# print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

# # Reshape trainX for LSTM input (samples, timesteps, features)
# TrainX = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))  # Assuming one feature for LSTM
# # TestX = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
# TrainY = y_train.reshape((y_train.shape[0],y_train.shape[1], 1))
# # TestY = y_test.reshape(y_test.shape[0])
# print(TrainX.shape,TrainY.shape)
# print(y_test)

# **Models**

In [ ]:
# # Define the LSTM model

def model_1 (input_shape, dropout_rate, units, output_size):
    model = Sequential()
    model.add(LSTM(units, return_sequences=True, input_shape=(input_shape, 1)))
    # model.add(Dropout(dropout_rate))
    model.add(LSTM(units))
    # model.add(Dropout(dropout_rate))
    # model.add(LSTM(units))
    # model.add(Dropout(dropout_rate))
    model.add(Dense(output_size, activation='relu'))  # Output layer with the shape of trainY
    # Compile the model
    model.compile(optimizer='Adam', loss='mean_squared_error', metrics=[MeanSquaredError()])
    model.summary()
    return model


input_shape=dataX.shape[1]
Dropout_rate = 0.2
units = 182
output_size= dataY.shape[1]
model= model_1(input_shape,Dropout_rate, units, output_size)
print("model 1 is running...")


# #%%


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 182, 182)            │         133,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 182)                 │         265,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 52)                  │           9,516 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 409,188 (1.56 MB)

 Trainable params: 409,188 (1.56 MB)

 Non-trainable params: 0 (0.00 B)

model 1 is running...


In [ ]:

# def model_2(input_shape , units, output_size):
#     model = Sequential()
#     model.add(LSTM(units, input_shape = (input_shape, 1)))
#     # model.add(Dropout(0.2))
#     model.add(Dense(output_size, activation ='relu'))
#     model.add(Dense(output_size, activation = 'linear'))
#     model.summary()
#     optimizer = tf.keras.optimizers.SGD(momentum=0.9)

#     model.compile(loss = MeanSquaredError(), metrics=['accuracy'], optimizer = optimizer)
#     return model


# input_shape=TrainX.shape[1]
# Dropout_rate = 0.2
# units=64
# output_size= y_train.shape[1]
# model= model_2(input_shape, Dropout_rate, units, output_size)
# print("model 2 is running...")


In [ ]:
# #    FROM LSTM_INSAT_161024

def model_3(n_steps_in , n_features, n_steps_out, e):


    model = Sequential()
    model.add(LSTM(26*7, activation='tanh', return_sequences=True, input_shape=(n_steps_in, n_features)))
    model.add(Dropout(0.2))
    model.add(LSTM(26*7, activation='tanh'))
    model.add(Dropout(0.2))
    model.add(Dense(n_steps_out,'relu'))
    opt=Adam()
    model.compile(optimizer=opt, loss=MeanSquaredError(), metrics= [MeanSquaredError()])
    model.summary()
    return model

model = model_3(n_past , n_features, n_future, e)
print("model 3 is running...")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 182, 182)            │         133,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 182, 182)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 182)                 │         265,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 182)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 78)                  │          14,274 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 413,946 (1.58 MB)

 Trainable params: 413,946 (1.58 MB)

 Non-trainable params: 0 (0.00 B)

model 3 is running...


In [ ]:
def model_4(n_steps_in , n_features, n_steps_out, e):

    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(26*7, activation='tanh', recurrent_activation='sigmoid', use_bias=True, input_shape=(n_steps_in, n_features), return_sequences=True),
        tf.keras.layers.LSTM(26*7, activation='tanh', recurrent_activation='sigmoid', use_bias=True, input_shape=(n_steps_in, n_features)),
        tf.keras.layers.Dense(n_steps_out, 'relu')
    ])
    learning_rate = 0.0012

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

   # Set the training parameters
    model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics= [keras.metrics.MeanSquaredError()])
    model.summary()
    return model

model = model_4(n_past , n_features, n_future, e)
print("model 4 is running...")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 182, 182)            │         133,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 182)                 │         265,720 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 78)                  │          14,274 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 413,946 (1.58 MB)

 Trainable params: 413,946 (1.58 MB)

 Non-trainable params: 0 (0.00 B)

model 4 is running...


# **Model Tuning**

In [ ]:
# def build_model(hp):
#     model = Sequential()
#     model.add(LSTM(
#         units=hp.Int('units', min_value=32, max_value=256, step=32),
#         activation='tanh',
#         return_sequences=True,
#         input_shape=(dataX.shape[1], 1)
#     ))
#     model.add(LSTM(
#         units=hp.Int('units', min_value=32, max_value=256, step=32),
#         activation='tanh',
#         return_sequences=False
#     ))
#     model.add(Dense(hp.Int('dense_units', min_value=16, max_value=128, step=16), activation='relu'))
#     model.add(Dense(1))

#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(
#             learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
#         ),
#         loss='mean_squared_error',
#         metrics=['mean_squared_error']
#     )
#     return model

# tuner = kt.RandomSearch(
#     build_model,
#     objective='val_loss',
#     max_trials=10,
#     executions_per_trial=2,
#     directory='lstm_tuning',
#     project_name='time_series'
# )

# tuner.search(dataX, dataY, epochs=50, validation_split=0.2, callbacks=[
#     tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
# ])


In [ ]:
# #Loading the weights of the model
#     model.load_weights(model_path)

#     #Using the Loaded model to predict values and scaling it
#     yhat = model.predict(input_array)
#     yhat_unscaled = (yhat*max_ins - min_ins) + min_ins

#     #To view the predictions made by the model
#     yhat_unscaled_x = yhat_unscaled.flatten()

# **Training**

In [ ]:

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
cp1 = ModelCheckpoint('model_lstm/best_model.keras', save_best_only=True) # Changed filepath to include filename and .keras extension# Fit the model
history = model.fit(dataX_scaled, dataY_scaled, epochs=e, batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping,reduce_lr,cp1])

plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training loss')
# plt.plot(history.history['val_loss'], label='Validation loss')
plt.plot(history.history['mean_squared_error'], label='Mean Square Error')
# plt.plot(history.history['RootMeanSquaredError'], label='RMSE')
plt.legend()

# Validaion
plt.figure(figsize=(10, 5))
plt.plot(history.history['val_loss'], label='Validation loss')
# plt.plot(history.history['val_loss'], label='Validation loss')
plt.plot(history.history['val_mean_squared_error'], label='Validation MSE')
# plt.plot(history.history['RootMeanSquaredError'], label='RMSE')
plt.legend()

NameError: name 'EarlyStopping' is not defined

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training loss')
# plt.plot(history.history['val_loss'], label='Validation loss')
plt.plot(history.history['mean_squared_error'], label='Mean Square Error')
# plt.plot(history.history['RootMeanSquaredError'], label='RMSE')
plt.legend()

# Validaion
plt.figure(figsize=(10, 5))
plt.plot(history.history['val_loss'], label='Validation loss')
# plt.plot(history.history['val_loss'], label='Validation loss')
plt.plot(history.history['val_mean_squared_error'], label='Validation MSE')
# plt.plot(history.history['RootMeanSquaredError'], label='RMSE')
plt.legend()

NameError: name 'history' is not defined

<Figure size 1000x500 with 0 Axes>

# **SAVE MODEL**

In [ ]:
# model.save('/content/drive/MyDrive/MTech Internship/my_model_m1.h5')

# model.save('/content/drive/MyDrive/MTech Internship/my_model_m3.h5')

model.save('/content/drive/MyDrive/MTech Internship/my_model_m4_2.h5')

model.save_weights('model_weights_m4.weights.h5')

model.save('/content/drive/MyDrive/MTech Internship/my_model_m4.keras') # Changed the file path to include the desired format .keras or .h5


NameError: name 'model' is not defined

In [ ]:
# model = load_model('/content/drive/MyDrive/MTech Internship/my_model_m1.h5')

# model = load_model('/content/drive/MyDrive/MTech Internship/my_model_m3.h5',
#                    custom_objects={'mean_squared_error': mean_squared_error})

model = load_model('/content/drive/MyDrive/MTech Internship/my_model_m4.h5')

# **Evaluation**

In [ ]:
dataX.shape
print(dataX[1][:][:].shape)

(182,)


In [ ]:

print(dataX_t.shape)
print(dataY_t)
# dataX_sam = dataX[0].reshape(1,dataX[0].shape[0],1)
# dataY_sam = dataY[0].reshape(1,dataY[0].shape[0],1)
# print(dataX_sam.shape)
# print(dataY_sam.shape)
results = model.evaluate(dataX_test_scaled, dataY_test_scaled)
print(f"Test Loss: {results[0]}, Test MSE: {results[1]}")



(520, 182)
[[ 57.358334  156.105     268.32834   ...  48.045       3.9533334
    0.       ]
 [156.105     268.32834   390.70834   ...   3.9533334   0.
   62.855    ]
 [268.32834   390.70834   515.705     ...   0.         62.855
  145.37666  ]
 ...
 [ 59.336666    4.74        0.        ... 457.385     326.55334
   62.203335 ]
 [  4.74        0.         93.69666   ... 326.55334    62.203335
   36.993332 ]
 [  0.         93.69666   212.745     ...  62.203335   36.993332
    4.488333 ]]
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0130 - mean_squared_error: 0.0259
Test Loss: 0.012956549413502216, Test MSE: 0.025914214551448822


# **Prediction**

In [ ]:
# print(dataX_t)
# print(dataY_t)
predictions = model.predict(dataX_test_scaled)
predictions = scalerY_test.inverse_transform(predictions)
# dataX_t = scaler.inverse_transform(dataX_t)
# dataY_t = scaler.inverse_transform(dataY_t)
print(predictions[1])
print(predictions[1].shape)

print(dataY_t)
print(dataY_t.shape)

# print(dataX_t[1])
# print(dataX_t[1].shape)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
[151.1404   230.02708    0.       402.0597   548.50745  637.59424
 738.23474  822.0675   898.2199     0.       964.4289   973.9801
 930.5201   881.56323  803.56854  695.6785   606.2015   495.89325
 396.75534  261.4354   171.63301   93.5801    81.6441    48.284214
  34.683167  76.19574  113.18799  209.38278  304.1844   387.6162
 518.28986  630.13684  710.0536   825.674    861.3251   932.5978
 951.02637  962.03503  888.53705  874.1201   818.42017  692.99854
 583.88367  471.73392  361.30182  293.42096  155.71036  115.16373
  73.00195   13.736274  31.673843  68.78369  125.586136 209.30551
 291.34274  419.63165  510.3481   634.0009   715.68463  810.8713
 837.3825   905.50305  930.2406   920.38513  873.298    826.4901
 750.128    674.68243  581.18384  491.99295  349.265    233.20529
 142.20699  106.13916   37.514645  24.917057  21.512028  81.704506]
(78,)
[[ 57.358334  156.105     268.32834   ...  48.045       3.9533334
    0.       ]
 [156.105     268

In [ ]:
df = pd.DataFrame({'Actuals': dataY_t[1].flatten(), 'Predictions': predictions[1]})

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/MTech Internship/predictions.csv', index=False)

In [ ]:
dataY_t.shape

(520, 78)

In [ ]:
# Calculate RMSE

# dataY_t_rescaled = scalerY_test.inverse_transform(dataY_t.reshape(dataY_t.shape[0],dataY_t.shape[1]))

print(predictions.shape)
rmse = root_mean_squared_error(dataY_t.reshape(-1), predictions.reshape(-1))
print("Root Mean Squared Error:", rmse)


# calculate MAE
mae = np.mean(np.abs(dataY_t[1] - predictions[1]))
print("Mean Absolute Error (MAE):", mae)

# # Calculate MAPE
# mape = np.mean(np.abs((dataY_t[1] - predictions[1]) / dataY_t[1])) * 100
# print("Mean Absolute Percentage Error (MAPE):", mape)

(520, 78)
Root Mean Squared Error: 151.86592311807502
Mean Absolute Error (MAE): 97.0172620380764


# **Plots**

In [ ]:

# Split data into two days
day1_predictions = predictions[1][:26]
day2_predictions = predictions[1][26:]



day1_test_y = dataY_t[1][:26]
day2_test_y = dataY_t[1][26:]
# day1_test_y = scaler.inverse_transform(day1_test_y)
# day2_test_y = scaler.inverse_transform(day2_test_y)

# Plot the data
plt.figure(figsize=(12, 6))

# Day 1
plt.subplot(2, 1, 1)
plt.plot(day1_predictions, label='Predicted')
plt.plot(day1_test_y, label='Actual')
plt.title('Day 1')
plt.xlabel('Time')
plt.ylabel('GHI')
plt.legend()

# Day 2
plt.subplot(2, 1, 2)
plt.plot(day2_predictions, label='Predicted')
plt.plot(day2_test_y, label='Actual')
plt.title('Day 2')
plt.xlabel('Time')
plt.ylabel('GHI')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Split data into two days
TwoDays_predictions = predictions[1][:78]
# day2_predictions = predictions[1][26:]



TwoDays_test_y = dataY_t[1][:78]
# day2_test_y = dataY_t[1][26:]
# TwoDays_test_y = scaler.inverse_transform(TwoDays_test_y)
# day2_test_y = scaler.inverse_transform(day2_test_y)

print(TwoDays_predictions.shape)
print(TwoDays_test_y.shape)
# Plot the data
plt.figure(figsize=(12, 6))

# Day 1
plt.subplot(2, 1, 1)
plt.plot(TwoDays_predictions, label='Predicted')
plt.plot(TwoDays_test_y, label='Actual')
plt.title('3 Days')
plt.xlabel('Time')
plt.ylabel('GHI')
plt.legend()

(78,)
(78,)


In [ ]:
# predictions = predictions.reshape(-1, 1)
# Y_test = Y_test.reshape(-1, 1)

In [ ]:

# Assuming p_df is your DataFrame with 'date_time' and prediction/actual columns

# Convert the 'date_time' column to a list (limiting to length of prediction)
dates = df_t['date_time'].tolist()[:len(predictions)]
# print(dates)
# Ensure predictions and Y_test are 1-dimensional and the same length
predictions = predictions.flatten()[:len(dates)]
dataY_t = dataY_t.flatten()[:len(dates)]

# Create the plot
plt.figure(figsize=(20, 10))
plt.plot(dates, predictions, label='Predicted')
plt.plot(dates, dataY_t, label='Actual')

# Customize the plot
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Predicted vs. Actual Values')
plt.legend()
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.grid(True)
plt.tight_layout()
plt.show()

NameError: name 'predictions' is not defined